In [1]:
import pandas as pd
import datetime
import re
import requests
import bs4

# Web Scraping - Build little functions

### I. Get suppe

In [2]:
def get_suppe(url):
    resp = requests.get(url)
    return bs4.BeautifulSoup(resp.content, "html.parser")

In [3]:
url = "https://www.coursereport.com/best-coding-bootcamps"
suppe = get_suppe(url)

### II. Get list of school items

In [80]:
def get_school_list(soup):
    return (soup
            .find("ul", id="schools")
            .find_all("li"))

In [12]:
school_list = get_school_list(suppe)

In [16]:
sample = school_list[14]

### III. For each school, extract

#### 1. rank and name

In [41]:
def get_rank(school_item):
    rank_pattern = r"^(\d{1,2})\."
    rank = (school_item
            .find("h3")
            .find("a")
            .text)
    return int(re.findall(rank_pattern, rank)[0])

def get_name(school_item):
    name_pattern = r"^\d{1,2}\.\s(.+)"
    name = (school_item
            .find("h3")
            .find("a")
            .text)
    return re.findall(name_pattern, name)[0]

In [42]:
get_rank(sample)

15

In [22]:
get_name(sample)

'Ironhack'

#### 2. Get rating

In [43]:
def get_rating(school_item):
    rating_pattern = r"\((.+)\)"
    rating = (school_item
              .find("span",
                    class_="longform-rating-text")
              .text)
    
    return float(re.findall(rating_pattern, rating)[0])

In [44]:
get_rating(sample)

4.88

#### 3. Get no of reviews

In [45]:
def get_reviews(school_item):
    reviews_pattern = r"(^\d*)\s"
    reviews = (school_item
               .find_all("span",
                         class_="longform-rating-text")[1]
               .find("a")
               .text)
    return int(re.findall(reviews_pattern, reviews)[0])

In [46]:
get_reviews(sample)

888

#### 4. Get Locations

---

In [33]:
"|".join(["A", "2", "c"])

'A|2|c'

In [34]:
pd.DataFrame([{"columnA": 12,
               "columnB": "ABC",
               "columnC": "|".join(["A", "2", "c"])}])

,columnA,columnB,columnC
0,12,ABC,A|2|c


---

In [35]:
def get_locations(school_item):
    location_list = (school_item
                     .find("span",
                           class_="location")
                     .find_all("a"))
    return "|".join([loc.text for loc in location_list])

In [36]:
get_locations(sample)

'Miami|Madrid|Mexico City|Paris|Berlin|Amsterdam|Sao Paulo|Lisbon|Barcelona'

#### 5. Get description

In [37]:
def get_description(school_item):
    return (school_item
            .find("div",
                  class_="desc-container")
            .find_all("p"))[1].text

In [38]:
get_description(sample)

'Ironhack offers 9-week, full-time and 24-week, part-time bootcamps in web development, UX/UI design, and data analytics in Miami (Florida), Madrid and Barcelona (Spain), Paris (France) Mexico\xa0City (Mexico), Berlin (Germany), Amsterdam (Netherlands),\xa0Sao Paulo (Brazil) and Lisbon (Portugal).\xa0Ironhack uses a customized approach to education by allowing students to shape their experience based on personal goals. Students who graduate from the Web Development Bootcamp will be skilled in technologies like\xa0JavaScript, HTML5 and CSS3.\xa0The UX/UI program covers Design Thinking, Photoshop, Sketch, Balsamiq, InVision, and JavaScript.\xa0Data Analytics covers data wrangling/cleaning, APIs, web scraping, and intermediate topics in Git, MySQL,\xa0Python, Data visualization, Panda, and Machine Learning. And the Data Analytics program allows students to load, clean, explore and extract valuable insights from datasets and cultivate languages scuh as Python, SQL and Tableau.\xa0 '

In [53]:
def get_stars(school_item):
    stars_dict = {"icon-full_star": 1,
                  "icon-half_star": .5,
                  "icon-empty_star": 0}
    stars = (school_item
             .find("div",
                   class_="ratings title-rating")
             .find_all("span"))[1:]
    
    return float(sum([stars_dict[star["class"][0]] for star in stars]))

In [48]:
get_stars(sample)

5.0

### IV. Create a function that processes one school_item

In [49]:
def get_record(school_item):
    return {"rank": get_rank(school_item),
            "name": get_name(school_item),
            "rating": get_rating(school_item),
            "stars": get_stars(school_item),
            "reviews": get_reviews(school_item),
            "locations": get_locations(school_item),
            "description": get_description(school_item)}

In [50]:
get_record(sample)

{'rank': 15,
 'name': 'Ironhack',
 'rating': 4.88,
 'stars': 5.0,
 'reviews': 888,
 'locations': 'Miami|Madrid|Mexico City|Paris|Berlin|Amsterdam|Sao Paulo|Lisbon|Barcelona',
 'description': 'Ironhack offers 9-week, full-time and 24-week, part-time bootcamps in web development, UX/UI design, and data analytics in Miami (Florida), Madrid and Barcelona (Spain), Paris (France) Mexico\xa0City (Mexico), Berlin (Germany), Amsterdam (Netherlands),\xa0Sao Paulo (Brazil) and Lisbon (Portugal).\xa0Ironhack uses a customized approach to education by allowing students to shape their experience based on personal goals. Students who graduate from the Web Development Bootcamp will be skilled in technologies like\xa0JavaScript, HTML5 and CSS3.\xa0The UX/UI program covers Design Thinking, Photoshop, Sketch, Balsamiq, InVision, and JavaScript.\xa0Data Analytics covers data wrangling/cleaning, APIs, web scraping, and intermediate topics in Git, MySQL,\xa0Python, Data visualization, Panda, and Machine Lea

### V. Process all schools

In [55]:
school_ranking = [get_record(school) for school in school_list]

In [56]:
df = pd.DataFrame(school_ranking)

In [57]:
df

,rank,name,rating,stars,reviews,locations,description
0,1,Flatiron School,4.71,4.5,435,Brooklyn|Houston|London|Austin|Denver|Seattle|...,Flatiron School offers immersive on-campus and...
1,2,Hack Reactor,4.69,4.5,268,Austin|Los Angeles|Denver|Seattle|Phoenix|Boul...,"Founded in 2012, Hack Reactor is a 12-week imm..."
2,3,Codesmith,4.90,5.0,347,Los Angeles|New York City|Online,"Codesmith offers a full-time, 12-week full sta..."
3,4,App Academy,4.68,4.5,799,New York City|Online|San Francisco,App Academy offers immersive web development c...
4,5,Turing,4.77,4.5,158,Denver,Turing School of Software & Design is a 7-mont...
5,6,Fullstack Academy,4.89,5.0,267,Jacksonville|San Diego|San Luis Obispo|New Yor...,Fullstack Academy offers full-time and part-ti...
6,7,General Assembly,4.32,4.5,439,Singapore|Orlando|Detroit|Raleigh|Miami|San Di...,General Assembly offers short and long courses...
7,8,Tech Elevator,4.95,5.0,203,Cincinnati|Pittsburgh|Philadelphia|Detroit|Col...,Tech Elevator is an immersive 14-week coding b...
8,9,DigitalCrafts,4.88,5.0,207,Kennesaw|Houston|Seattle|Online|Atlanta,DigitalCrafts offers a 16-week full-time and a...
9,10,Software Guild,4.66,4.5,146,Minneapolis|Louisville|Online|Atlanta,"The Software Guild offers immersive full-time,..."


### VI. Adjusting the functions to account for the date the data was pulled

In [62]:
print(datetime.date.today())

2020-04-06


In [83]:
def get_record(school_item, bootcamp_type, date=datetime.date.today()):
    return {"date_id": date,
            "bootcamp_type": bootcamp_type,
            "rank": get_rank(school_item),
            "name": get_name(school_item),
            "rating": get_rating(school_item),
            "stars": get_stars(school_item),
            "reviews": get_reviews(school_item),
            "locations": get_locations(school_item),
            "description": get_description(school_item)}

In [86]:
def get_rankings(school_list, bootcamp_type, date=datetime.date.today()):
    return pd.DataFrame([get_record(school, bootcamp_type, date) for school in school_list])

In [87]:
get_rankings(school_list, "coding")

,date_id,bootcamp_type,rank,name,rating,stars,reviews,locations,description
0,2020-04-06,coding,1,Flatiron School,4.71,4.5,435,Brooklyn|Houston|London|Austin|Denver|Seattle|...,Flatiron School offers immersive on-campus and...
1,2020-04-06,coding,2,Hack Reactor,4.69,4.5,268,Austin|Los Angeles|Denver|Seattle|Phoenix|Boul...,"Founded in 2012, Hack Reactor is a 12-week imm..."
2,2020-04-06,coding,3,Codesmith,4.90,5.0,347,Los Angeles|New York City|Online,"Codesmith offers a full-time, 12-week full sta..."
3,2020-04-06,coding,4,App Academy,4.68,4.5,799,New York City|Online|San Francisco,App Academy offers immersive web development c...
4,2020-04-06,coding,5,Turing,4.77,4.5,158,Denver,Turing School of Software & Design is a 7-mont...
5,2020-04-06,coding,6,Fullstack Academy,4.89,5.0,267,Jacksonville|San Diego|San Luis Obispo|New Yor...,Fullstack Academy offers full-time and part-ti...
6,2020-04-06,coding,7,General Assembly,4.32,4.5,439,Singapore|Orlando|Detroit|Raleigh|Miami|San Di...,General Assembly offers short and long courses...
7,2020-04-06,coding,8,Tech Elevator,4.95,5.0,203,Cincinnati|Pittsburgh|Philadelphia|Detroit|Col...,Tech Elevator is an immersive 14-week coding b...
8,2020-04-06,coding,9,DigitalCrafts,4.88,5.0,207,Kennesaw|Houston|Seattle|Online|Atlanta,DigitalCrafts offers a 16-week full-time and a...
9,2020-04-06,coding,10,Software Guild,4.66,4.5,146,Minneapolis|Louisville|Online|Atlanta,"The Software Guild offers immersive full-time,..."


In [90]:
def get_rankings(url, bootcamp_type, date=datetime.date.today()):
    soup = get_suppe(url)
    school_list = get_school_list(soup)
    return pd.DataFrame([get_record(school, bootcamp_type, date) for school in school_list])

In [91]:
data_science_url = "https://www.coursereport.com/best-data-science-bootcamps"
get_rankings(data_science_url, "data-science")

,date_id,bootcamp_type,rank,name,rating,stars,reviews,locations,description
0,2020-04-06,data-science,1,BrainStation,4.68,4.5,332,Vancouver|Toronto|New York City|Online,BrainStation offers full-time and part-time co...
1,2020-04-06,data-science,2,Coding Temple,4.91,5.0,114,Dallas|Online|Washington|Boston|Chicago,"Coding Temple offers 10-week, full-stack codin..."
2,2020-04-06,data-science,3,Divergence Academy,5.00,5.0,20,Dallas,Divergence Academy is a 12-week full-time data...
3,2020-04-06,data-science,4,Flatiron School,4.71,4.5,435,Brooklyn|Austin|Atlanta|Houston|Seattle|London...,Flatiron School offers immersive on-campus and...
4,2020-04-06,data-science,5,Galvanize,4.47,4.5,177,Boulder|Austin|Seattle|New York City|Los Angel...,Galvanize offers a 13-week full-time and a 26-...
5,2020-04-06,data-science,6,General Assembly,4.32,4.5,439,Sydney|Austin|Atlanta|Singapore|Melbourne|Dall...,General Assembly offers short and long courses...
6,2020-04-06,data-science,7,Metis,4.89,5.0,100,Seattle|New York City|Online|Chicago|San Franc...,Metis offers data science training via 12-week...
7,2020-04-06,data-science,8,NYC Data Science Academy,4.84,5.0,287,New York City|Online,NYC Data Science Academy offers 12-week data s...
8,2020-04-06,data-science,9,Principal Analytics Prep,4.88,5.0,17,New York City,"Principal Analytics Prep offers a 12-week, ful..."
9,2020-04-06,data-science,10,Propulsion Academy,5.00,5.0,54,Zurich,Propulsion Academy offers 12-week full-time bo...


In [93]:
list_of_bootcamps = ["coding", "data-science", "online"]


In [92]:
def get_rankings(bootcamp_type, date=datetime.date.today()):
    url = f"https://www.coursereport.com/best-{bootcamp_type}-bootcamps"
    soup = get_suppe(url)
    school_list = get_school_list(soup)
    return pd.DataFrame([get_record(school, bootcamp_type, date) for school in school_list])

In [96]:
coding, data_science, online = [get_rankings(bc) for bc in list_of_bootcamps]

In [97]:
online

,date_id,bootcamp_type,rank,name,rating,stars,reviews,locations,description
0,2020-04-06,online,1,AcadGild,4.30,4.0,172,Bangalore|Online,AcadGild is an online coding bootcamp offering...
1,2020-04-06,online,2,Actualize,4.88,5.0,234,Online|Chicago,Actualize is a 12-week software development bo...
2,2020-04-06,online,3,Altcademy,5.00,5.0,48,Hong Kong|Online,Altcademy (formerly Hack Pacific) is an educat...
3,2020-04-06,online,4,Bloc,4.66,4.5,449,Online,Bloc is an online coding bootcamp that incorpo...
4,2020-04-06,online,5,Bottega,4.48,4.5,60,Salt Lake City|Online,Bottega offers part-time and full-time tech bo...
5,2020-04-06,online,6,CareerFoundry,4.54,4.5,356,Berlin|Online,CareerFoundry is an online coding school that ...
6,2020-04-06,online,7,Coding Dojo,4.43,4.5,432,Tulsa|Boise|Orange County|Silicon Valley|Arlin...,Coding Dojo is a unique coding bootcamp that t...
7,2020-04-06,online,8,Covalence,4.66,4.5,50,Online,"Covalence provides career-changing 12-week, fu..."
8,2020-04-06,online,9,Designlab,4.68,4.5,104,Online,Designlab teaches in-demand UX/UI design skill...
9,2020-04-06,online,10,Flatiron School,4.71,4.5,435,Brooklyn|Houston|Austin|New York City|London|A...,Flatiron School offers immersive on-campus and...


### Question: Is Ironhack in the data science list?

In [102]:
"Ironhack" in data_science["name"].to_list()

False